In [36]:
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import json
from torch import tensor
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import label_binarize

mne.set_log_level('ERROR')

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import argparse
import os
gpus = [0]
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, gpus))
import numpy as np
import math
import glob
import random
import itertools
import datetime
import time
import datetime
import sys
import scipy.io
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

'''from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchsummary import summary
import torch.autograd as autograd
from torchvision.models import vgg19'''

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn.init as init

from torch.utils.data import Dataset
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
# from common_spatial_pattern import csp
import torch
from mamba_ssm import Mamba

In [18]:
class PatchEmbedding(nn.Module):
    def __init__(self, emb_size=40):
        # self.patch_size = patch_size
        super().__init__()

        self.shallownet = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), (1, 1)),
            nn.Conv2d(40, 40, (22, 1), (1, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 75), (1, 15)),  # pooling acts as slicing to obtain 'patch' along the time dimension as in ViT
            nn.Dropout(0.5),
        )

        self.projection = nn.Sequential(
            nn.Conv2d(40, emb_size, (1, 1), stride=(1, 1)),  # transpose, conv could enhance fiting ability slightly
            Rearrange('b e (h) (w) -> b (h w) e'),
        )


    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        x = self.shallownet(x)
        x = self.projection(x)
        return x

torch.Size([1, 77, 40])


In [37]:
import torch
from mamba_ssm import Mamba


d_model=40
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=d_model, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")


In [42]:
optimal_threshold=.34 #actual=.087
def decode_predictions(predictions):
    agregated_outputs=predictions.mean(dim=1)
    probabilities=F.softmax(agregated_outputs,dim=-1)
    #print(probabilities)
    
    if probabilities[0][1]>optimal_threshold:
        return 'preictal'
    else:
        max_values, max_indices = torch.max(probabilities, dim=1)
        if max_indices==0:
            return 'interictal'
        else:
            return 'ictal'
    

In [21]:
raw = mne.io.read_raw_edf("../CHB-MIT/chb02_19.edf", preload=True)

def preprocessing(raw):

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


    freqs = (60,120)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
    
    raw.notch_filter(freqs=freqs, picks=eeg_picks)  # Apply notch filter
    raw.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    numpy_array=raw.get_data()
    segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
    return(segment_tensor)

In [22]:
CHB_files={'../CHB-MIT/chb02_19.edf': [(0, 2769, 'interictal'), (2769, 3369, 'preictal'), (3369, 3378, 'ictal'), (3378, 3600, 'interictal')],}
true_labels=[]
start,end=2778,3388
for i in range(start,end):
    for interval in CHB_files.values():
        for start_interval,end_interval,label in interval:
            if start_interval <= i < end_interval:
                true_labels.append(label)
                break
        else:
            continue
        break
    


In [ ]:
CNNmodel=PatchEmbedding().to(device)
model=Mamba(d_model=40).to(device)

segment_tensor=preprocessing(raw)
predictions=[]

with torch.no_grad():  # Disable gradient computation for inference

    for i in range(2778,3388):
        input=segment_tensor[:,:,:,256*i:256*(i+1)].float().to(device)
        cnn_output=CNNmodel(input)
        print(cnn_output.shape)
        output=model(cnn_output)
        output.mean(axis=1)
        



        predicted_label_names=decode_predictions(output)
        predictions.append(predicted_label_names)

In [45]:
print(predictions)

['ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 'ictal', 

In [ ]:
input= torch.rand(1,1,28,256).float().

